# Milestone II:  Implementation

Group Member: Camilla Ren, Mina Yang, Xiwen Wei

## Data Prepocressing

#### 1) Loading Dataset

In [25]:
# Load packages
import pandas as pd
import altair as alt
import ast

In [26]:
# Load dataset
url = 'https://raw.githubusercontent.com/shlrley/amazon_bestsellers/main/data/amazon_conlit_goodreads_nyt.csv'
amazon_book_original = pd.read_csv(url,parse_dates=['amazon_year'])
amazon_book_original.head()

,title,amazon_author,amazon_rating,amazon_num_reviews,amazon_price,amazon_year,amazon_genre,conlit_genre,conlit_pubdate,conlit_author_gender,...,goodreads_publish_date,goodreads_first_publish_date,goodreads_awards,goodreads_num_ratings,goodreads_likedPercent,goodreads_price,nyt_published_date,nyt_list_name_encoded,nyt_price,nyt_weeks_on_list
0,11/22/63: A Novel,Stephen King,4.6,2052,22,2011-01-01,Fiction,BS,2011-01-01,M,...,2011-11-08,NaN,['Locus Award Nominee for Best SF Novel (2012)...,420225.0,96.0,6.21,2012-02-26,hardcover-fiction,35.00,14.0
1,A Dance with Dragons (A Song of Ice and Fire),George R. R. Martin,4.4,12643,11,2011-01-01,Fiction,BS,2011-01-01,M,...,2011-07-12,NaN,"['Hugo Award Nominee for Best Novel (2012)', '...",555900.0,97.0,NaN,NaN,NaN,NaN,NaN
2,A Gentleman in Moscow: A Novel,Amor Towles,4.7,19699,15,2017-01-01,Fiction,BS,2016-01-01,M,...,2019-03-26,2016-09-06,"['Kirkus Prize Nominee for Fiction (2016)', 'G...",293330.0,96.0,6.29,2019-12-29,trade-fiction-paperback,0.00,34.0
3,A Stolen Life: A Memoir,Jaycee Dugard,4.6,4149,32,2011-01-01,Non Fiction,MEM,2011-01-01,F,...,2011-07-12,2011-07-11,['Goodreads Choice Award Nominee for Memoir & ...,96524.0,92.0,3.32,2011-10-02,hardcover-nonfiction,24.99,10.0
4,All the Light We Cannot See,Anthony Doerr,4.6,36348,14,2014-01-01,Fiction,PW,2014-01-01,M,...,2014-05-06,2014-05-28,"['Pulitzer Prize for Fiction (2015)', 'Audie A...",1024442.0,96.0,4.45,2016-04-10,hardcover-fiction,0.00,99.0


#### 2) Data Wrangling

In [27]:
# Drop the unused columns
amazon_book = amazon_book_original.drop(columns=['amazon_year', 'conlit_pubdate','conlit_author_nationality', 'conlit_total_ratings',
                                                 'goodreads_rating', 'goodreads_series', 'goodreads_edition','goodreads_publisher',
                                                 'goodreads_publish_date', 'goodreads_first_publish_date', 'goodreads_awards', 'goodreads_num_ratings','goodreads_likedPercent',
                                                 'goodreads_price', 'nyt_published_date','nyt_list_name_encoded','nyt_price','goodreads_genres']) 

In [28]:
# Print each of the unique elements in conlit_author_gender column
amazon_book['conlit_author_gender'].unique()

array(['M', 'F', nan], dtype=object)

In [29]:
# Print each of the unique elements in conlit_genre column
amazon_book['conlit_genre'].unique()

array(['BS', 'MEM', 'PW', 'MID', 'MIX', 'HIST', nan, 'BIO', 'SF', 'YA',
       'NYT'], dtype=object)

In [30]:
# Replace the N/A value in conlit_author_gender and conlit_genre columns with "Other"
amazon_book['conlit_author_gender'] = amazon_book['conlit_author_gender'].fillna("Other")
amazon_book['conlit_genre'] = amazon_book['conlit_genre'].fillna("Other")

In [31]:
#Remove missing values
amazon_book = amazon_book.dropna()

In [32]:
# Print out the column names, their data types, and count of non-NA values of cleaned dataset
amazon_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 365
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 169 non-null    object 
 1   amazon_author         169 non-null    object 
 2   amazon_rating         169 non-null    float64
 3   amazon_num_reviews    169 non-null    int64  
 4   amazon_price          169 non-null    int64  
 5   amazon_genre          169 non-null    object 
 6   conlit_genre          169 non-null    object 
 7   conlit_author_gender  169 non-null    object 
 8   nyt_weeks_on_list     169 non-null    float64
dtypes: float64(2), int64(2), object(5)
memory usage: 13.2+ KB


## Visulization for 5 tasks

### Task 1: Browse the prices between fiction and nonfiction books. (Browse)

Attributes: title, amazon_price, amazon_genre

In [33]:
alt.Chart(amazon_book).mark_circle().encode(
  alt.X('amazon_price:O'),
  alt.Y('count()'),
  alt.Size('count()'),
  alt.Color('amazon_genre', scale=alt.Scale(range=['red', 'green']))
)

alt.Chart(...)

### Task 2:  Who is the most popular (female/male) author in the last few years (with highest reviews)?  (Search)

Attributes: amazon_author, amazon_num_reviews, conlit_author_gender

In [34]:
selectTitle = alt.selection_single(
    name='Select', 
    fields=['title']
)

In [35]:
plot_1 = alt.Chart(amazon_book).mark_bar().encode(
    alt.Y('title', sort='-x'),
    alt.X('amazon_num_reviews:Q'),
    alt.Color('conlit_author_gender'),
    alt.Tooltip(['amazon_author', 'title']),
    opacity=alt.condition(selectTitle, alt.value(0.8), alt.value(0.1))
).properties(height = 180, width = 280, title = "Bar Chart of Number of Reviews").transform_filter(
    'datum.amazon_num_reviews > 33000'
).add_selection(
    selectTitle
)
plot_1

alt.Chart(...)

In [36]:
highest_reviews_book = ["The Girl on the Train", "The Nightingale: A Novel", "Fifty Shades of Grey: Book One of the Fifty Shades Trilogy (Fifty Shades of Grey Series)",
                       "Where the Crawdads Sing", "All the Light We Cannot See", "The Goldfinch: A Novel (Pulitzer Prize for Fiction)", "The Hunger Games (Book 1)",
                       "The Alchemist"]

### Task 3: Which genre of book has the highest rating? （Search)

Attributes: goodreads_genres, amazon_rating

In [37]:
order = amazon_book.groupby('conlit_genre')['amazon_rating'].median().sort_values(ascending = False).index.to_list()

alt.Chart(amazon_book).mark_boxplot().encode(
  alt.Y('conlit_genre:O', sort = order),
  alt.X('amazon_rating:Q', scale=alt.Scale(zero=False)),
  alt.Color('conlit_genre')
).transform_filter('isValid(datum.conlit_genre)').properties(height = 200)

alt.Chart(...)

### Task 4:Which book is the best seller? (Query)

Attributes: nyt_weeks_on_list, title

In [38]:
base = alt.Chart(amazon_book).encode(
    alt.Theta("nyt_weeks_on_list", stack = True),
    alt.Color("title:N")
).properties(height = 180, width = 280, title = "Pie Chart of Number of Weeks").transform_filter(
    'datum.nyt_weeks_on_list > 288'
)

pie = base.mark_arc(outerRadius=90)
text = base.mark_text(radius=100, size=15).encode(text="nyt_weeks_on_list:Q")

plot_3 = pie + text
plot_3

alt.LayerChart(...)

In [39]:
highest_weeks_book = ["Wrecking Ball (Diary of a Wimpy Kid Book 14)", "The Alchemist", 
                      "Fantastic Beasts and Where to Find Them: The Original Screenplay (Harry Potter)",
                      "Old School (Diary of a Wimpy Kid #10)", "Diary of a Wimpy Kid: The Long Haul",
                      "Diary of a Wimpy Kid: The Long Haul",
                     "Harry Potter Paperback Box Set (Books 1-7)",
                     "Diary of a Wimpy Kid: Hard Luck, Book 8"]

### Task 5: Is there a big price difference between science fiction and non-science fiction books? (Query)

Attributes: amazon_genre, amazon_price

In [40]:
plot_b = alt.Chart(amazon_book).transform_density(
    'amazon_price',
    groupby=['amazon_genre'],
    as_=['amazon_price', 'density'],
).mark_area(
    opacity=0.6
).encode(
    alt.X('amazon_price:Q'),
    alt.Y('density:Q', stack=None), #'zero'
    alt.Color('amazon_genre:N'),
)

## Visulization Implementation

### View 1: task 1 + task 5

In [41]:
click = alt.selection_multi(on='mouseover', fields=['amazon_genre'], bind='legend')

plot_a = alt.Chart(amazon_book).mark_circle().encode(
    alt.X('amazon_price:O', title='Price'),
    alt.Y('count()', title='Count'),
    alt.Size('count()'),
    alt.Color('amazon_genre', 
              scale=alt.Scale(range=['red', 'green']), 
              title='Genre'),
    opacity=alt.condition(click, alt.value(0.6), alt.value(0.2))
).properties(
    width=550,
    height=300
)

plot_b = alt.Chart(amazon_book).transform_density(
    'amazon_price',
    groupby=['amazon_genre'],
    as_=['amazon_price', 'density']
).mark_area(
    opacity=0.6
).encode(
    alt.X('amazon_price:Q', title='Price'),
    alt.Y('density:Q', stack=None, title='Density'),
    alt.Color('amazon_genre:N', title='Genre'),
    alt.Tooltip('amazon_price'),
    opacity=alt.condition(click, alt.value(0.6), alt.value(0.2))
).properties(
    width=550,
    height=300
).add_selection(click)

alt.vconcat(plot_b, plot_a).properties(title='Amazon Book Prices and Genres')

alt.VConcatChart(...)

### View 2: task 2 & task 4

In [42]:
plot_4 = alt.Chart(amazon_book).mark_bar(opacity = 0.8).encode(
    alt.Y('title', sort='-x'),
    alt.X('nyt_weeks_on_list:Q'),
    alt.Color('conlit_author_gender'),
    alt.Tooltip(['amazon_author', 'title']),
    opacity=alt.condition(selectTitle, alt.value(0.8), alt.value(0.1))
).properties(height = 180, width = 280, title = "Bar Chart of Number of Weeks").transform_filter(
    'datum.nyt_weeks_on_list > 288'
).add_selection(
    selectTitle
)

plot_4

alt.Chart(...)

In [43]:
amazon_book_filtered1 = amazon_book[amazon_book['title'].isin(highest_reviews_book)]
amazon_book_review1 = amazon_book_filtered1.loc [:, ['title','amazon_num_reviews', 'conlit_author_gender', 'amazon_author', 'nyt_weeks_on_list']]
amazon_book_review1 = amazon_book_review1.drop_duplicates()

In [44]:
plot_2 = alt.Chart(amazon_book_review1).mark_bar().encode(
    alt.Y('title', sort=highest_reviews_book),
    alt.X('nyt_weeks_on_list:Q', scale=alt.Scale(domain=(0, 600))),
    alt.Color('conlit_author_gender'),
    alt.Tooltip(['amazon_author', 'title']),
    opacity=alt.condition(selectTitle, alt.value(0.8), alt.value(0.1))
).properties(height = 180, width = 280, title = "Bar Chart of Number of Weeks").add_selection(
    selectTitle
)

In [45]:
(plot_1 & plot_3).resolve_scale(color='independent') | (plot_2 & plot_4)

alt.HConcatChart(...)

### View 3: task 3

In [46]:
order = amazon_book.groupby('conlit_genre')['amazon_rating'].median().sort_values(ascending = False).index.to_list()

rating_boxplot = alt.Chart(amazon_book).mark_boxplot().encode(
  alt.Y('conlit_genre:O', title='Literary Genre',sort = order),
  alt.X('amazon_rating:Q', title='Amazon Rating',scale=alt.Scale(zero=False)),
).transform_filter(
    'isValid(datum.conlit_genre)'
).properties(
    width=450,
    height=300
)


select_genre = alt.selection_single(
    name='Select', fields=['conlit_genre'], 
    bind=alt.binding_select(options=list(amazon_book['conlit_genre'].unique()))
)


zoom_selection = alt.selection_interval(bind='scales')


query = alt.Chart(amazon_book).mark_square(size=80).encode(
    x=alt.X('amazon_rating:Q', title='Amazon Rating',scale=alt.Scale(zero=False)),
    y=alt.Y('conlit_genre:N', title='Literary Genre',sort = order),
    color=alt.condition(select_genre, alt.Color('amazon_genre:N', 
                        scale=alt.Scale(range=['#e377c2', '#2ca02c'])) , alt.value('#cccccc')),
    tooltip=['title', 'amazon_author', 'amazon_rating', 'amazon_num_reviews'],
    opacity=alt.condition(select_genre, alt.value(1), alt.value(0.1))
).properties(
    width=450,
    height=300
).add_selection(select_genre, zoom_selection)

alt.concat(rating_boxplot,query).properties(title='Compare the rating for each genre of book')



alt.ConcatChart(...)